# Travelling SalesPerson and OpenStreetMap

### Goals for today
* Learn how to load and handle data from osm (and viewing it with contextily)
* Compute the travelling salesperson on real data using techniques we already know

In [ ]:
# %pip install networkx
# %pip install matplotlib
# %pip install scipy
# %pip install numpy

In [ ]:
# %pip install scikit-learn

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy.linalg as la
import numpy as np

## Loading in data from OSM

To access Open Street Map we can use osmnx which converts the map to a networkx graph.

To view the map (often described as 'tiles') we use contextily, which gives us access to a wide range of different maps.

In [ ]:
# %pip install osmnx
# %pip install contextily

In [ ]:
import osmnx as ox
import contextily

We can retrieve data from OSM from a point:

In [ ]:
G = ox.graph.graph_from_point((49.26653, -123.25552), dist=1000, network_type="walk")

In [ ]:
ox.plot_graph(G)

Or from a bounding box:

In [ ]:
north = 49.282126
south = 49.242523
east = -123.227730
west = -123.267649

G = ox.graph.graph_from_bbox(north=north, south=south, east=east, west=west, network_type='walk')

In [ ]:
ox.plot_graph(G)

For this project we are going to look at just the 'walk' network type. Other types include 'bike' and 'drive'.

We can take a look at the nodes and edges by converting them into GeoDataFrames

In [ ]:
nodes, edges  = ox.graph_to_gdfs(G)

In [ ]:
edges.shape,nodes.shape

In [ ]:
edges.head()

In [ ]:
edges.columns

In [ ]:
nodes.head()

We can also view the network using contextily. (Make sure the Coordinate Reference System is the same!)

In [ ]:
def plot_graph(f,ax,edges):
    edges.plot(linewidth=.25, ax=ax, color='k')
    contextily.add_basemap(ax=ax,
                        crs=edges.crs,
                        source = contextily.providers.CartoDB.Voyager)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
f,ax = plt.subplots(1,1, figsize=(10,10))
plot_graph(f,ax,edges)

## Travelling Sales Person

Let $V$ be a set of nodes and let $c_{ij}$ be the cost (or distance) to travel from node $i$ to node $j$. A **tour** is a sequence of nodes such that each node is visited exactly once. The cost of a tour is the sum of the costs of each edge $c_{ij}$ in the tour including the last step to return to the initial node. The TSP (or routing problem) is to find the tour with the minimum cost.

In [ ]:
north = 49.278126
south = 49.250523
east = -123.240730
west = -123.267649

bbox = (north,south,east,west)

G = ox.graph.graph_from_bbox(bbox = bbox, network_type='walk')

nodes, edges  = ox.graph_to_gdfs(G)

Let's create our list of nodes that we want to travel to.

In [ ]:
lsk = (49.26546608185259, -123.25539661804837)
ikb = (49.26760838969576, -123.25262857840771)
wreck_beach = (49.26235090542639, -123.26156798293921)
nest = (49.2669443748462, -123.25012905864469)
botannical_gardens = (49.2544735954425, -123.25082054453766)
bus_stop = (49.265994305701106, -123.24819721938316)
important_points = [
    lsk,
    ikb,
    wreck_beach,
    nest,
    botannical_gardens,
    bus_stop
]
important_point_names = [
    'LSK',
    'IKB',
    'Wreck Beach',
    'Nest',
    'Botannical Gardens',
    'Bus Loop'
]

In [ ]:
f,ax = plt.subplots(1,1, figsize=(10,10))
plot_graph(f,ax,edges)
for point in important_points:
    ax.scatter(point[1],point[0],s=100)

We can use the same methods that we worked with in the previous class:

In [ ]:
def cost_matrix(V):
    n = V.shape[0]
    C = np.zeros((n,n))
    for i in range(1,n):
        for j in range(0,i):
            C[i,j] = la.norm(V[i,:] - V[j,:])
            C[j,i] = C[i,j]
    return C

def tour_cost(C,tour):
    n = len(tour)
    cost = 0
    for i in range(n-1):
        cost += C[tour[i],tour[i+1]]
    cost += C[tour[-1],tour[0]]
    return cost

def nearest_neighbor(V,start=0, C = None):
    n = V.shape[0]
    if C is None:
        C = cost_matrix(V)
    tour = [start]
    nodes = list(range(n))
    nodes.remove(start)
    for i in range(1,n):
        next_i = np.argmin(C[tour[-1],nodes])
        next_node = nodes[next_i]
        tour.append(next_node)
        nodes.pop(next_i)
    cost = tour_cost(C,tour)
    return tour,cost

In [ ]:
V = np.array(important_points)
tour,cost = nearest_neighbor(V)

In [ ]:
tour_named = [important_point_names[i] for i in tour]
tour_named

In [ ]:
f,ax = plt.subplots(1,1, figsize=(10,10))
plot_graph(f,ax,edges)
for point in important_points:
    ax.scatter(point[1],point[0],s=100)
for i in range(len(V)-1):
    ax.plot([V[tour[i],1],V[tour[i+1],1]],[V[tour[i],0],V[tour[i+1],0]],'b',linewidth=1)
ax.plot([V[tour[-1],1],V[tour[0],1]],[V[tour[-1],0],V[tour[0],0]],'b',linewidth=1)
plt.show()

But, this isn't how we walk! We have to keep to our network. To do this we must rethink the costs of travelling between two nodes on our network.

If we are travelling from node $i$ to node $j$ we don't need to consider any other nodes. We want to minimize our path length between the two nodes and therefore the costs should be the shortest path between the nodes.

Let's convert our important points to the node closest to them on the network.

In [ ]:
important_nodes = [ox.distance.nearest_nodes(G, point[1],point[0]) for point in important_points]

We also need to add the weights to the graph, based on the edge lengths.

In [ ]:
G = ox.distance.add_edge_lengths(G)

We compute the shortest paths:

In [ ]:
shortest_paths = [[nx.shortest_path(G,node_0,node_1,weight='length') for node_1 in important_nodes] for node_0 in important_nodes]
shortest_path_lengths = np.array([[nx.shortest_path_length(G,node_0,node_1,weight='length') for node_1 in important_nodes] for node_0 in important_nodes])


In [ ]:
def get_path_edges(path_nodes,edges):
    edge_list = [(path_nodes[j],path_nodes[j+1]) for j in range(len(path_nodes)-1)]
    return edges[edges.index.isin(edge_list)]

def plot_paths(f,ax,tour,shortest_paths):
    for i in range(len(tour)-1):
        shortest_path_edges = get_path_edges(shortest_paths[tour[i]][tour[i+1]],edges)
        shortest_path_edges.plot(linewidth=3, ax=ax, color='b')
    shortest_path_edges = get_path_edges(shortest_paths[tour[-1]][tour[0]],edges)
    shortest_path_edges.plot(linewidth=3, ax=ax, color='b')

Should we run the nearest neighbour algorithm or the brute force method? How many different tours are possible?

In [ ]:
from scipy.special import factorial
n = len(important_points)

We can run the brute force algorithm with the updated costs:

In [ ]:
from itertools import permutations

def brute_force(V,C=None):
    n = V.shape[0]
    tours = permutations(range(n))
    if C is None:
        C = cost_matrix(V)
    optimal_cost = None
    for tour in tours:
        cost = tour_cost(C,tour)
        if optimal_cost is None or cost < optimal_cost:
            optimal_cost = cost
            optimal_tour = np.array(tour)
    return optimal_tour,optimal_cost

In [ ]:
tour,cost = brute_force(V,C=shortest_path_lengths)

In [ ]:
tour_named = [important_point_names[i] for i in tour]
tour_named

In [ ]:
f,ax = plt.subplots(1,1, figsize=(10,10))
plot_graph(f,ax,edges)
for point in important_points:
    ax.scatter(point[1],point[0],s=100)
plot_paths(f,ax,tour,shortest_paths)
plt.show()

We can also run the nearest neighbor algorithm, though this may have some stranger results.

In [ ]:
tour,cost = nearest_neighbor(V,C=shortest_path_lengths)
tour_named = [important_point_names[i] for i in tour]
tour_named

In [ ]:
f,ax = plt.subplots(1,1, figsize=(10,10))
plot_graph(f,ax,edges)
for point in important_points:
    ax.scatter(point[1],point[0],s=100)
plot_paths(f,ax,tour,shortest_paths)
plt.show()

Some questions to discuss and explore:
* Now that we have edge weights, can we also play around with network flows? What routes should you take to avoid crowds of people going to the bus stops?
* Can we do the same thing (TSP and Network flows) with roads? How could we define the edge weights?
* We can access Wreck Beach at multiple locations. How can we include this in the TSP?
